# ***Data Analysis***
#***YAHOO Finance Stock dataset Statistical analysis - Laptev Oleg, gr.4167, RCSE-GRIAT***

> Working with CSV format Yahoo Finance Inc. dataset made for the dash-demo application

> Using: *Matplotlib,Pandas,Statsmodels API,ARIMA,ANOVA,K-MEANS,Regression and Classification models, Clustering*





**Preparing the data**

In [ ]:
!pip3 install pmdarima

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

In [ ]:
pip install yfinance

In [ ]:
df = pd.read_csv(r'/content/drive/My Drive/prices.csv')
df

In [ ]:
df['date']

In [ ]:
df

In [ ]:
from datetime import datetime
con=df['date']
df['date']=pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
#check datatype of index


In [ ]:
df=df.sort_values(by='date')

In [ ]:
df

In [ ]:
df1=df.loc[df['symbol']=='WMT']


In [ ]:
df1

In [ ]:
dfopen=df1['open']


In [ ]:

df1['year'] = df1.index.year
df1['month'] = df1.index.month
df1['day'] = df1.index.day



> Display a random sampling of 5 rows



In [ ]:

df1.sample(5, random_state=0)

In [ ]:
df.columns = df.columns.get_level_values(0)

In [ ]:
df.columns

In [ ]:
df.columns = [' '.join(col).strip() for col in df.columns.values]

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
temp=df1.groupby(['date'])['open'].mean() 
temp.plot(figsize=(15,5), title= 'Opening Prices(Monthwise)', fontsize=14)

In [ ]:
df1.groupby('month')['open'].mean().plot.bar()

In [ ]:
dfopen=df1['open'].loc[:'20140530']

In [ ]:
histogram=dfopen.describe()
dfopen.hist()
histogram

In [ ]:
test = dfopen[1030:]
train = dfopen[:1029]
train.plot(style='b-')
test.plot(style='r--')

In [ ]:
def test_stationarity(timeseries):
 #Determing rolling statistics
 rolmean = timeseries.rolling(12).mean()
 rolstd = timeseries.rolling(12).std()
 #Plot rolling statistics:
 plt.plot(timeseries, color='blue',label='Original')
 plt.plot(rolmean, color='orange', label='Rolling mean')
 plt.plot(rolstd, color='black', label = 'Rolling std')
 plt.legend(loc='Best')
 plt.title('Rolling Mean and Standard Deviation')
 plt.show(block=False)
 
 print("Results of Dickey-Fuller ")
 adftest = adfuller(timeseries,autolag='AIC')
 # output for adft will give us without defining what the values are.
 #hence we manually write what values does it explain using a for loop
 output = pd.Series(adftest[0:4],index=['Test stat.','p-value','Num. of lags used','Num of observations used'])
 for key,values in adftest[4].items():
  output['critical value (%s)'%key] = values
 print(output)



> Test the data by algorithm



In [ ]:
plt.figure(figsize=(12,14))

test_stationarity(train)

**Critical value, the p-value is greater than 5%, and we can see an increasing trend in the data. It means that data is not stationary.**
> *Hence, we have to stationarize it*



In [ ]:
plt.figure(figsize=(12,14))

train_log = np.log(train) 
test_log = np.log(test)
moving_avg = train_log.rolling(24).mean() 
plt.plot(train_log) 
plt.plot(moving_avg, color = 'red') 
plt.show()



> We can observe that there is a trend, so it is not stationary. Now we will remove this trend to make our time series stationary.



In [ ]:
train_log_moving_average_diff = train_log - moving_avg

In [ ]:
plt.figure(figsize=(12,14))

train_log_moving_average_diff.dropna(inplace = True), test_stationarity(train_log_moving_average_diff)



> Now, the Test Statistic is less than the Critical Value and the p-value is less than 5%. So, we can be confident that the trend is almost removed.



![alt text](https://)

> ***Now  we should stabilize the variance, b.c. it is also a requirement of stationary time series***




In [ ]:
plt.figure(figsize=(12,14))

train_log_diff = train_log - train_log.shift(1) 
test_stationarity(train_log_diff.dropna())



> Now we will decompose the time series into trend and seasonality and will get the residual which is the random variation in the series.

We are removing the seasonal part 



In [ ]:
from pmdarima import auto_arima
model = auto_arima(train_log, trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train_log)
forecast = model.predict(n_periods=len(test))
forecast = pd.DataFrame(forecast,index = test_log.index,columns=['Prediction'])
#plot the predictions for validation set
plt.plot(train_log, label='Train')
plt.plot(test_log, label='Test')
plt.plot(forecast,'g--', label='Prediction')
plt.title('Wallmart Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Actual Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()



> We will use the RMSE(Root Mean Square Error) to judge our forecast results



In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
rms = sqrt(mean_squared_error(test_log,forecast))
print("RMSE: ", rms)

In [ ]:
# Calculate the absolute errors
errors = abs(np.ravel(forecast) - np.ravel(test_log))

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Determine Performance Metrics
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (np.ravel(errors) / np.ravel(test_log))

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Prediction Accuracy is:', round(accuracy, 2), '%.')

**Well, 1.1% of error is a VERY GOOD result in our case**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import gmean

In [ ]:
price_df = pd.read_csv('/content/drive/My Drive/prices.csv')
sec_df = pd.read_csv('/content/drive/My Drive/securities.csv')
fund_df = pd.read_csv('/content/drive/My Drive/fundamentals.csv')

In [ ]:
plt.figure(figsize=(15, 6))
ax = sns.countplot(y='GICS Sector', data=sec_df)
plt.xticks(rotation=45)

In [ ]:
sec_df = sec_df.rename(columns = {'Ticker symbol' : 'symbol','GICS Sector' : 'sector'})
sec_df.head()

In [ ]:
price_df  = price_df.merge(sec_df[['symbol','sector']], on = 'symbol')
price_df['date'] = pd.to_datetime(price_df['date'])
price_df.head()

In [ ]:
price_df = price_df[price_df['date'] <= '2015-12-15']

In [ ]:
sector_pivot = pd.pivot_table(price_df, values = 'open', index = ['date'],columns = ['sector']).reset_index()
sector_pivot

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(sector_pivot.corr('kendall'),annot=True, cmap="coolwarm")

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(sector_pivot.corr('spearman'),annot=True, cmap="coolwarm")

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(sector_pivot.corr('pearson'),annot=True, cmap="coolwarm")

In [ ]:
#importing lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go
# init_notebook_mode()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""

    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

In [ ]:
x, y = ecdf(price_df['open'])

samples = np.random.normal(np.mean(price_df['open']), np.std(price_df['open']), size=10000)
x_theor, y_theor = ecdf(samples)
sns.set()
plt.figure(figsize=(12,14))
plt.plot(x_theor, y_theor)

plt.plot(x, y, marker=".", linestyle="none")

plt.legend(('Normal Distribution', 'Opening costs empirical Data'), loc='lower right')

In [ ]:
x, y = ecdf(price_df['close'])

samples = np.random.normal(np.mean(price_df['close']), np.std(price_df['close']), size=10000)
x_theor, y_theor = ecdf(samples)
sns.set()
plt.figure(figsize=(12,14))
plt.plot(x_theor, y_theor)

plt.plot(x, y,'go')

plt.legend(('Normal Distribution', 'Closing costs empirical Data'), loc='lower right')

In [ ]:
x, y = ecdf(price_df['volume'])

samples = np.random.normal(np.mean(price_df['volume']), np.std(price_df['volume']), size=10000)
x_theor, y_theor = ecdf(samples)
sns.set()
plt.figure(figsize=(12,14))
plt.plot(x_theor, y_theor)

plt.plot(x, y,'r--')

plt.legend(('Normal Distribution', 'Costs volume empirical Data'), loc='lower right')

In [ ]:
x, y = ecdf(price_df['high'])

samples = np.random.normal(np.mean(price_df['high']), np.std(price_df['high']), size=10000)
x_theor, y_theor = ecdf(samples)
sns.set()
plt.figure(figsize=(12,14))
plt.plot(x_theor, y_theor)

plt.plot(x, y,'y.')

plt.legend(('Normal Distribution', 'Highest costs empirical Data'), loc='lower right')

In [ ]:
x, y = ecdf(price_df['low'])

samples = np.random.normal(np.mean(price_df['low']), np.std(price_df['low']), size=10000)
x_theor, y_theor = ecdf(samples)
sns.set()
plt.figure(figsize=(12,14))
plt.plot(x_theor, y_theor,'r-')

plt.plot(x, y,'b.')

plt.legend(('Normal Distribution', 'Least costs empirical Data'), loc='lower right')

In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from math import sqrt

Hence, we can see that all costs data features are distributed closely to the Normal Gaussian rule


In [ ]:
stocks = fund_df

In [ ]:
top_rev = stocks.groupby(by='Ticker Symbol').agg({'Total Revenue':sum})

In [ ]:
g = top_rev['Total Revenue'].nlargest(5)

In [ ]:
next = [Bar(
            y=g,
            x=g.keys(),
            marker = dict(
            color = 'lightsteelblue'
            ),
            name = "Contractor's amount earned per project"
    )]
layout1 = go.Layout(
    title="Top 10 Exporters",
    xaxis=dict(
        title='Company',
        titlefont=dict(
            size=30,
            color='#7f7f7f'
               )
    ),
    yaxis=dict(
        title='Total Revenue',
        titlefont=dict(
            size=22,
            color='#7f7f7f'
        )
    )
)
myFigure2 = go.Figure(data = next, layout = layout1)
iplot(myFigure2)

Here are the top 5 stocks that has the biggest revenue: 

    1. Walmart 
    2. EXXON MOBIL corp
    3. Apple inc.
    4. Chevron Corp.
    5. General Motoros

In [ ]:
WMT = stocks[stocks['Ticker Symbol']=='WMT']
#sns.distplot(Aqua['Generation'],bins=28,kde=False,color='red')

In [ ]:
gir = ['Total Equity',
       'Total Revenue',
       'Accounts Payable',
       'Accounts Receivable',
      'Cost of Revenue',
      'Profit Margin',
      'Sale and Purchase of Stock',
      'Earnings Per Share',
       'Net Borrowings']
tip = np.corrcoef(WMT[gir].values.T)

In [ ]:
WMT.info()

In [ ]:
sns.set(font_scale = 0.9)
plt.figure(figsize=(12,14))

map = sns.heatmap(tip, cbar = True,
                  cmap="YlGnBu",
                  annot = True, 
                  square= True,
                  fmt = '.1f',
                  annot_kws = {'size':11}, 
                 yticklabels = gir,
                 xticklabels = gir)

#  **Linear Regression**

In [ ]:

n = WMT['Total Revenue']
m = WMT[['Total Equity',
       'Accounts Payable',
       'Accounts Receivable',
      'Cost of Revenue',
      'Profit Margin',
      'Sale and Purchase of Stock',
      'Earnings Per Share',
       'Net Borrowings']]
target=['Total Revenue']
features = ['Total Equity',
       'Accounts Payable',
       'Accounts Receivable',
      'Cost of Revenue',
      'Profit Margin',
      'Sale and Purchase of Stock',
      'Earnings Per Share',
       'Net Borrowings']

In [ ]:
# Splitting the into sets of training and test.
train,test,train_label,test_label=train_test_split(m,n,test_size=0.33,random_state=101)

In [ ]:
lr = LinearRegression()
lr.fit(train, train_label)
# coef_ - is an array of features
print(np.ravel(lr.coef_))

In [ ]:
Linear = LinearRegression(fit_intercept=True)
mo = Linear.fit(train,train_label)
predi = mo.predict(test)
print(r2_score(test_label,predi))

In [ ]:
print('Correlation: ', round(np.corrcoef(np.ravel(test_label), np.ravel(predi))[0,1], 5))
fig = plt.figure(figsize=(10, 6))
plt.title('Correlation between predicted and actual results (Linear Regressor)')
plt.plot(test_label, predi, 'r*')
plt.xlabel('actual')
plt.ylabel('predicted')
plt.show()

In [ ]:
# Finding the coefficient. (value of 1 unit increase) 
coef = pd.DataFrame(Linear.coef_,m.columns,columns=['Coefficient'])
coef

In [ ]:
errors = abs(np.ravel(predi) - np.ravel(test_label))

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Determine Performance Metrics
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (np.ravel(errors) / np.ravel(test_label))

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

# **Polynomial regression**

In [ ]:
# Polynomial regression
# Fit on train set
model = PolynomialFeatures(degree=2)
X_train_ = model.fit_transform(test)
X_test_ = model.fit_transform(test)

plr = LinearRegression()
plr.fit(X_train_, train_label)
predicted_data = plr.predict(X_test_)


# predicted_data = np.round_(predicted_data)
# correct result; round of prediction, prediction


In [ ]:
print('R^2: ', plr.score(X_test_, test_label))

In [ ]:
RMSE = sqrt(mean_squared_error(y_true=test_label, y_pred=predicted_data))
print('RMSE: ', RMSE)

In [ ]:
print('squared errors: ', 
      round(sum(np.ravel(abs(test_label - np.around(predicted_data)))) / len(test_label), 3))

In [ ]:
errors = abs(np.ravel(predicted_data) - np.ravel(test_label))

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
mape = 100 * (np.ravel(errors) / np.ravel(test_label))

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Polynomial Regression Accuracy:', round(accuracy, 2), '%.')

# **Random Forest**

In [ ]:
# Train Model
# Instantiate model 
rfg = RandomForestRegressor(n_estimators= 1000, random_state=42, criterion = 'mse', max_depth = None,
                            min_samples_split = 2, min_samples_leaf = 1)

# Train the model on training data
rfg.fit(train, np.ravel(train_label));

# Make Predictions on Test Data
# Use the forest's predict method on the test data
y_prediction = rfg.predict(test)



In [ ]:
print('Correlation: ', round(np.corrcoef(np.ravel(test_label), np.ravel(y_prediction))[0,1], 5))
fig = plt.figure(figsize=(10, 6))
plt.title('Correlation between actual and predicted results (Random Forest Regressor)')
plt.plot(test_label, y_prediction, 'r*')
plt.xlabel('actual')
plt.ylabel('predicted')
plt.show()

In [ ]:
print('R^2: ', rfg.score(test, test_label))

In [ ]:
RMSE = sqrt(mean_squared_error(y_true=test_label, y_pred=y_prediction))
print('RMSE: ', RMSE)

In [ ]:
print('squared errors: ', 
      round(sum(np.ravel(abs(np.ravel(test_label) - np.around(y_prediction)))) / len(test_label), 3))

In [ ]:
errors = abs(np.ravel(y_prediction) - np.ravel(test_label))

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
# Determine Performance Metrics
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (np.ravel(errors) / np.ravel(test_label))

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
tree = rfg.estimators_[50]
print('The depth of this tree is:', tree.tree_.max_depth)

In [ ]:
# Variable Importances
# Get numerical feature importances
importances = list(rfg.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(m, round(importance, 2)) for m, importance in zip(m, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
x_values = list(range(len(importances)))

# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')

# Tick labels for x axis
plt.xticks(x_values, features, rotation='vertical')

# Axis labels and title
plt.ylabel('Importance'); plt.title('Feature Importances');

In [ ]:
# A cumulative importance graph: shows the contribution to the overall importance 
# of each additional variable. 
# The dashed line - at 95% of total importance accounted for.
# After that some unimportant features (sulphates and may be density) can be removed.
# 95% - is an arbitrary threshold.
plt.figure(figsize=(12,14))
# List of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)
# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')
# Draw line at 95% of importance retained
plt.hlines(y = 0.95, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')
# Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')
# Axis labels and title
plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');

In [ ]:
# Find number of features for cumulative importance of 95%
# Add 1 because Python is zero-indexed
importantFeaturesCount = np.where(cumulative_importances > 0.95)[0][0] + 1
print('Count of features for 95% importance:', importantFeaturesCount)

# Extract the names of the most important features
important_feature_names = [feature[0] for feature in feature_importances[0:importantFeaturesCount]]
# Find the columns of the most important features
important_indices = [features.index(feature) for feature in important_feature_names]
# Create training and testing sets with only the important features
important_train_features = train.iloc[:, important_indices]
important_test_features = test.iloc[:, important_indices]
# Sanity check on operations
print('Important train features shape:', important_train_features.shape)
print('Important test features shape:', important_feature_names)

In [ ]:
important_feature_names

In [ ]:
# As it can be seen from the results, they became worse than for the case with all the features
# Removing the so-called "unimportant" feature did not improve metrics

# Training and Evaluating on Important Features
# Train the expanded model on only the important features
rfg.fit(important_train_features, np.ravel(train_label));
# Make predictions on test data
predictions = rfg.predict(important_test_features)
# Performance metrics
print('R^2: ', rfg.score(important_test_features, test_label))
errors = abs(np.ravel(predictions) - np.ravel(test_label))
print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (np.ravel(errors) / np.ravel(test_label))
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

# **Testing Classification prediction on different training samples**

In [ ]:
reg1 = GradientBoostingRegressor(random_state=1, n_estimators=10)
reg2 = RandomForestRegressor(random_state=1, n_estimators=10)
reg3 = LinearRegression()
ereg = VotingRegressor([('gb', reg1), ('rf', reg2), ('lr', reg3)])
reg1.fit(train, train_label.values.ravel())
reg2.fit(train, train_label.values.ravel())
reg3.fit(train, train_label.values.ravel())
ereg.fit(train, train_label.values.ravel())

# for some 20 values
xt = train[:20]

plt.figure(figsize=(10, 6))
plt.plot(reg1.predict(xt), 'gd', label='GradientBoostingRegressor')
plt.plot(reg2.predict(xt), 'b^', label='RandomForestRegressor')
plt.plot(reg3.predict(xt), 'ys', label='LinearRegression')
plt.plot(ereg.predict(xt), 'r*', label='VotingRegressor')
plt.tick_params(axis='x', which='both', bottom=False, top=False,
                labelbottom=False)
plt.ylabel('predicted')
plt.xlabel('training samples')
plt.legend(loc="best")
plt.title('Comparison of individual predictions with averaged')
plt.show()

In [ ]:
xt = test[:20]

plt.figure(figsize=(10, 6))
plt.plot(reg1.predict(xt), 'gd', label='GradientBoostingRegressor')
plt.plot(reg2.predict(xt), 'b^', label='RandomForestRegressor')
plt.plot(reg3.predict(xt), 'ys', label='LinearRegression')
plt.plot(ereg.predict(xt), 'r*', label='VotingRegressor')
plt.tick_params(axis='x', which='both', bottom=False, top=False,
                labelbottom=False)
plt.ylabel('predicted')
plt.xlabel('training samples')
plt.legend(loc="best")
plt.title('Comparison of individual predictions with averaged')
plt.show()

In [ ]:
# KNN
mainFrame = pd.read_csv('/content/drive/My Drive/prices.csv')

mainFrame["change"] = mainFrame["close"] - mainFrame["open"]

df_interest = mainFrame[["symbol", "date", "open", "close", "change", "volume"]]

df_interest["date"] = pd.to_datetime(df_interest["date"])

df_interest.head()

symbols = df_interest["symbol"].unique().tolist()

full_count = len(symbols)



oppFrame = df_interest.pivot(index = 'date', columns = 'symbol', values = 'close')

oppFrame = oppFrame.dropna(axis=1)

opp_symbols = oppFrame.columns

part_count = len(oppFrame.columns) 

'''

for u in opp_symbols[:10]:

    plt.plot(oppFrame.index.tolist(), oppFrame[u].values.tolist())

plt.legend(symbols, loc='upper left')

plt.show()

'''




ss = df_interest.groupby(by=["symbol"])["change"].std()

ss = (ss-ss.mean())/ss.std()



pcs = df_interest.groupby(by='symbol').apply(lambda grp: grp[grp['change'] > 0]['change'].count() / grp['change'].size)

pcs = (pcs-pcs.mean())/pcs.std()



avgv = df_interest.groupby(by=['symbol'])['volume'].mean()/10000000

avgv = (avgv-avgv.mean())/avgv.std()



newdf = pd.concat([ss, pcs, avgv], axis=1).reset_index()

newdf.columns = ['symbol', 'std', 'prop_pos_day_change', "avg_volume"]

newdf.head()



for i in newdf['symbol'].tolist():

    x = newdf[newdf['symbol'] == i]['std']

    y = newdf[newdf['symbol'] == i]['prop_pos_day_change']

    plt.scatter(x,y)

plt.legend(newdf['symbol'].tolist(),

           bbox_to_anchor=(1.05, 1),

           loc=2,

           borderaxespad=0.,

          ncol=10)



plt.title(r'Stock Change and Spread', fontsize=32)

plt.xlabel('Daily Positive Change (%)', fontsize=22)

plt.ylabel('Total Standard Devation', fontsize=22)

plt.figure(figsize=(10,100))

plt.show()



df1 = newdf.iloc[0:10,:]

df2 = newdf.iloc[100:110,:]

df3 = newdf.iloc[200:210,:]

df4 = newdf.iloc[300:310,:]

df5 = newdf.iloc[400:410,:]

testdf = pd.concat([df1,df2,df3,df4,df5])

for i in range(0,50):

    testdf.index.values[i] = i



from sklearn.cluster import KMeans

kmdf = testdf

met={}

# Visualize K = {3..9}

kValues = [i for i in range(3,10)]

for k in kValues:

    kmeans = KMeans(n_clusters=k, random_state=0).fit(kmdf[['std','prop_pos_day_change']].to_numpy())

    kmdf[str(k)] = kmeans.labels_



kmdf = pd.melt(kmdf, 

                id_vars=["symbol", 'std', 'prop_pos_day_change'],

                var_name="k", 

                value_name="values",

                value_vars=list(kmdf.columns[-7:]))



kmdf.head()



g = sns.FacetGrid(kmdf, col="k", hue="values", col_wrap=4, palette='Set2')

g = g.map(plt.scatter, "std", "prop_pos_day_change")

g.set(xlabel="Closing Deviation")

g.set(ylabel="'Daily Positive Change (%)")

g.fig.suptitle("Stock Cluster Analysis", size=28)

g.fig.subplots_adjust(top=.8)

plt.subplots_adjust(hspace=1.2, wspace=0.4)

g.add_legend()

g._legend.set_title("Cluster")

#handles = g._legend_data.values()

#labels = g._legend_data.keys()

#g.fig.legend(handles=handles, labels=labels, loc='lower right', ncol=3)

met={}
from sklearn import metrics

for i in range(4,10):

    met[str(i)] = metrics.silhouette_score(kmdf.loc[kmdf['k']==str(i)][['std','prop_pos_day_change']], kmdf.loc[kmdf['k']==str(i)]['values'], metric='euclidean')



metdf = pd.Series(met)

# **Factor Analysis with PCA**

In [ ]:
stocks.isnull().any()
newdata = pd.DataFrame(stocks[gir]) 
newdata

In [ ]:
X = newdata.interpolate()
X.shape

In [ ]:
X = X.dropna()
X.shape

In [ ]:
def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )

x_before_pca = pd.DataFrame(X)
x_before_pca.describe()
x_before_pca.shape
plot_correlation_map(x_before_pca)
from sklearn.decomposition import PCA as PCA
pca = PCA(n_components=9)
pca.fit(x_before_pca)
var = pca.explained_variance_ratio_
var1 = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
var1
plt.plot(var1)
x_pca = PCA(n_components=3)
x_pca.fit(x_before_pca)
x = x_pca.fit_transform(x_before_pca)
d = {'pc1': x[:,0], 'pc2': x[:, 1], 'pc3': x[:,2]}
x_df = pd.DataFrame(d)
x_df.head(3)
x_df.describe()
x_new_ndarray = x_pca.inverse_transform(x_df)
x_new = pd.DataFrame(x_new_ndarray)
x_new.columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']
x_new.head(3)
x_before_pca.head(3)
plt.scatter(x_df['pc1'], x_df['pc2'], color = 'green')


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs = x_df['pc1'], ys = x_df['pc2'], zs= x_df['pc3'], zdir='z')

In [ ]:
x_df.describe()


In [ ]:
x_df

In [ ]:
plt.scatter(x_df['pc1'], x_df['pc2'], color = 'red')


# **ANOVA**(**AN**alysis **O**f **VA**riance)

In [ ]:
# load packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# load data file
d = stocks
# d['Date Local']=pd.to_datetime(d['Date Local'])
# reshape the d dataframe suitable for statsmodels package 
# you do not need to reshape if your data is already in stacked format. Compare d and d_melt tables for detail 
# understanding 
d_melt = pd.melt(d, id_vars=['Total Revenue'], value_vars=['Total Equity',
       'Accounts Payable',
       'Accounts Receivable',
      'Cost of Revenue',
      'Profit Margin',
      'Sale and Purchase of Stock',
      'Earnings Per Share',
       'Long-Term Investments',
       'Net Borrowings'])
# replace column names

d_melt.columns = ['Revenue', 'features', 'value']
# generate a boxplot to see the data distribution by genotypes and years. Using boxplot, we can easily detect the 
# differences between different groups
sns.set(rc={'figure.figsize':(20.7,22.27)})
sns.scatterplot(x="Revenue", y="value", hue="features", data=d_melt)

In [ ]:
import scipy.stats as stats
from statsmodels.formula.api import ols
results = ols('value ~ C(Revenue) + C(features)',data=d_melt).fit()

In [ ]:
results.summary()

In [ ]:
import statsmodels.api as sm

anova_table = sm.stats.anova_lm(results, typ=2)
anova_table

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder


# **Classification**


In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(train)
X_test = sc.fit_transform(test)

In [ ]:

###
# Random Forest Classifier
# A random forest is a meta estimator that fits a number of decision tree classifiers on various
# sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.
# The sub-sample size is always the same as the original input sample size
# but the samples are drawn with replacement if bootstrap=True (default).
# n_estimators - The number of trees on the forest

rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(train, np.ravel(train_label))
pred_rfc = rfc.predict(test)
# print some values (correct and predicted ones)
print('correct:  ', np.ravel(test_label.values[0:15]))
print('predicted:', pred_rfc[0:15])

In [ ]:
for i in range(0,11):
    print('     ' + str(i), end='')
print('\n')
pred_prob = rfc.predict_proba(X_test)[0:15]
# print correct result and predicted probabilities for each value


In [ ]:
print('Correlation: ', round(np.corrcoef(np.ravel(test_label), np.ravel(pred_rfc))[0,1], 5))
fig = plt.figure(figsize=(10, 6))
plt.title('Correlation between predicted and actual results (Random Forest Classifier)')
plt.plot(test_label, pred_rfc, 'r*')
plt.xlabel('actual')
plt.ylabel('predicted')
plt.show()

In [ ]:
# Determine Performance Metrics
# Calculate mean absolute percentage error (MAPE)


Error = np.mean(np.abs(np.subtract(test_label,pred_rfc)))
Average = np.mean(test_label)
MAPE = (Error/Average)*100

print('MAPE=',MAPE,'%.')
print('Accuracy=',100-MAPE,'%.')

In [ ]:
###
# Stochastic Gradient Descent Classifier
# The advantages of SGD:
# - Efficiency.
# - Ease of implementation (lots of opportunities for code tuning).
# The disadvantages of SGD:
# - SGD requires a number of hyperparameters such as the regularization parameter and the number of iterations.
# - SGD is sensitive to feature scaling.

sgd = SGDClassifier(penalty="elasticnet", max_iter=2000, tol=0.00001, loss="modified_huber")
sgd.fit(X_train, np.ravel(train_label))
pred_sgd = sgd.predict(test)
# print(sgd.coef_)
# print some values (correct and predicted ones)
print('correct:  ', np.ravel(test_label.values[0:15]))
print('predicted:', np.ravel(pred_sgd[0:15]))

In [ ]:
print('Correlation: ', round(np.corrcoef(np.ravel(test_label), np.ravel(pred_sgd))[0,1], 5))
fig = plt.figure(figsize=(10, 6))
plt.title('Correlation between predicted and actual results (Stochastic Gradient Descent Classifier)')
plt.plot(test_label, pred_sgd, 'r*')
plt.xlabel('actual')
plt.ylabel('predicted')
plt.show()

In [ ]:

print(classification_report(test_label, pred_sgd))

print(confusion_matrix(test_label, pred_sgd))

In [ ]:
# Determine Performance Metrics
# Calculate mean absolute percentage error (MAPE)


Error = np.mean(np.abs(np.subtract(test_label,pred_sgd)))
Average = np.mean(test_label)
MAPE = (Error/Average)*100

print('MAPE=',MAPE,'%.')
print('Accuracy=',100-MAPE,'%.')

In [ ]:
###
# Support Vector Classifier
# fit time complexity is more than quadratic with the number of samples -> hard to scale more than 10000 samples.
# probability=True - to use predict_proba method

svc = SVC(probability=True)
svc.fit(train, np.ravel(train_label))
pred_svc = svc.predict(test)
# print some values (correct and predicted ones)
print('correct:  ', np.ravel(test_label.values[0:15]))
print('predicted:', pred_svc[0:15])

In [ ]:
print('Correlation: ', round(np.corrcoef(np.ravel(test_label), np.ravel(pred_svc))[0,1], 5))
fig = plt.figure(figsize=(10, 6))
plt.title('Correlation between predicted and actual results (Support Vector Classifier)')
plt.plot(test_label, pred_svc, 'r*')
plt.xlabel('actual')
plt.ylabel('predicted')
plt.show()

In [ ]:

print(classification_report(test_label, pred_svc))

print(confusion_matrix(test_label, pred_svc))
# Determine Performance Metrics
# Calculate mean absolute percentage error (MAPE)


Error = np.mean(np.abs(np.subtract(test_label,pred_svc)))
Average = np.mean(test_label)
MAPE = (Error/Average)*100

print('MAPE=',MAPE,'%.')
print('Accuracy=',100-MAPE,'%.')


In [ ]:
# draw a bar plot with comparison of accuracy for different classification algorithms

# prepare data
accuracyVals = [ 98.21, 98.83,98.83]
names = ['Stochastic Gradient Descent Classifier', 'Support Vector Classifier', 'Random Forest Classifier']

plt.figure(figsize=(10, 6))
plt.bar(names, accuracyVals)
plt.title('Classification algorithms accuracy comparison')
plt.xlabel('classification algorithm')
plt.ylabel('accuracy, %')

plt.show()

In [ ]:
# draw a bar plot with comparison of accuracy for different classification algorithms

# prepare data
accuracyVals = [99.13, 97.6, 98.38]
names = ['Linear Regression', 'Polynomial Regression', 'Random Forest Regressor',]

plt.figure(figsize=(10, 7))
plt.bar(names, accuracyVals)
plt.title('Regression algorithms accuracy comparison')
plt.xlabel('Regression algorithm')
plt.ylabel('accuracy, %')

plt.show()

# **What did we get?**



*  **Most important features**:

      1.   Total Equity
      2.   Net borrowings


*   **Cluster analysis**:

      * 9 stock changes clusters by K-Means
      * only 2 clusters from *fundamentals* analysis
      
      * Best Classifier:
          * Random Forest Classifier

*   **Best regression algorithm**
      *   Linear regression


*   **ANOVA**
      *  C (Revenue) **p-value**: 1.07*e-38;
      *  C (features) **p-value**: 2.52*e-38







